* Project: LSMS_ag_prod 
* Created on: Feb 2025
* Created by: rg
* Edited on: 26 Feb 25
* Edited by: rg

* does
	* create summary stats per wave for reach country

* assumes
	* access to aggregated data

* notes:
    * there is no project.do file so file paths have to be updated manually 

In [1]:
# create tables for each country showinf mean and sd per wave
# import libraries 

import pandas as pd

In [2]:
# specify file patht to the data 

path =  '/Users/rodrigoguerra/Library/CloudStorage/OneDrive-UniversityofArizona/weather_and_agriculture/lsms_base/countries/aggregate/allrounds_final_weather_cp.dta'
df = pd.read_stata(path)

# define file path to save tables
save_path = '/Users/rodrigoguerra/Library/CloudStorage/OneDrive-UniversityofArizona/ag_prod/graphs&tables/tables/descriptive_stats/'

/var/folders/wj/dcwc9sc12tz299sg285qbzvr0000gn/T/ipykernel_34846/2006345036.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(path)
/var/folders/wj/dcwc9sc12tz299sg285qbzvr0000gn/T/ipykernel_34846/2006345036.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(path)
/var/folders/wj/dcwc9sc12tz299sg285qbzvr0000gn/T/ipykernel_34846/2006345036.py:4: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of

In [3]:
# drop if "crop" is missing and specify variables to be included in the graph
df = df.dropna(subset = ["crop"])

selected_vars =  ["yield_USD", "total_labor_days", "fert_USD", "plot_area_GPS"]

In [4]:
# dictionary to store tables per country 
country_tables = {}

# dictionary to rename variables (not show default name eg. yield_USD)
rename_dict = {
    "yield_USD": "Yield",
    "total_labor_days": "Total Labor Days",
    "fert_USD": "Inorganic Fertilizer", 
    "plot_area_GPS": "Plot Size"
}

In [5]:
# create a loop to process each country separately 

for country in df["country"].unique():
    df_country = df[df["country"] == country]

    # calculate mean and sd per wave for selected vars
    summary = df_country.groupby("wave")[selected_vars].agg(['mean', 'std'])

    # transpose to get the variables as rows
    summary = summary.T.unstack(level = 1)

    # round to two decimal places
    summary = summary.round(2)

    # format index of table 
    new_columns = pd.MultiIndex.from_tuples([(f"Wave {wave}", stat.capitalize()) for wave, stat in summary.columns])
    summary.columns = new_columns

    # rename variables 
    summary.rename(index = rename_dict, inplace = True)

    # store table in dictionary 
    country_tables[country] = summary 

    # print to preview tables
    print(f"\nSummary Table for {country}:")
    print(summary)

    # save as lateX 
    summary.to_latex(f"{save_path}{country}_summary_stats.tex", 
                 index=True, 
                 caption=f"Summary Statistics for {country}", 
                 label=f"tab:{country}_summary", 
                 multicolumn=True, 
                 multicolumn_format="c", 
                 column_format="l" + " c c" * (len(summary.columns) // 2),
                 float_format=lambda x: "{:,.2f}".format(x)) 



Summary Table for Ethiopia:
                      Wave 1            Wave 2              Wave 3             \
                        Mean      Std     Mean        Std     Mean        Std   
Yield                 690.05  2619.06  6221.19  478246.45  9003.42  116912.77   
Total Labor Days       24.15    19.58    28.04      24.05    26.22      22.71   
Inorganic Fertilizer    0.45     0.48     2.51       6.46     2.61       6.23   
Plot Size               0.17     0.39     0.17       0.91     0.17       3.16   

                       Wave 4             Wave 5            
                         Mean       Std     Mean       Std  
Yield                 1153.19  17886.91  2405.32  22789.40  
Total Labor Days        27.73     22.01    21.62     17.53  
Inorganic Fertilizer     2.46      5.51     2.17      4.68  
Plot Size                0.14      0.20     0.12      0.23  

Summary Table for Malawi:
                      Wave 1          Wave 2           Wave 3          Wave 4  \
          